In [1]:
#Import required libraries
import cv2
import numpy as np
import math

In [2]:
def horizontalEdge(img):
    
    patch = [[-1,-2,-1],[0,0,0],[1,2,1]]        #Sobel horizontal edge detection filter
    patch = np.array(patch)                     # convert the patch to an array

    patch_size = [3,3]                          #define patch size 
    img_shape = img.shape                       #image size

    output_matrix = np.zeros([img_shape[0]-2, img_shape[1]-2])  #If image is of size m*n and patch p*q the resultanat of convoluton is of size m-p+1 * n-q+1
    for i in range(img_shape[0]-2):
        for j in range(img_shape[1]-2):
            output = np.zeros(patch_size)       #define a output patch
            for k in range(patch_size[0]):
                for m in range(patch_size[1]):
                    output[k,m]=img[i+k,j+m]    #to extract an output patch
            temp = np.sum(patch*output)         # calculate the sum of product of the output patch with the sobel edge detection filter
            temp = temp/8                       #Normalisation
            output_matrix[i,j]=temp
    return output_matrix

In [3]:
def verticalEdge(img):
    
    patch = [[1,0,-1],[2,0,-2],[1,0,-1]]        #Sobel vertical edge detection filter
    patch = np.array(patch)                     # convert the patch to an array

    patch_size = [3,3]                          #define patch size
    img_shape = img.shape                       #image size

    output_matrix = np.zeros([img_shape[0]-2, img_shape[1]-2])  #If image is of size m*n and patch p*q the resultanat of convoluton is of size m-p+1 * n-q+1
    for i in range(img_shape[0]-2):
        for j in range(img_shape[1]-2):
            output = np.zeros(patch_size)       #define a output patch
            for k in range(patch_size[0]):
                for m in range(patch_size[1]):
                    output[k,m]=img[i+k,j+m]    #to extract an output patch
            temp = np.sum(patch*output)         # calculate the sum of product of the output patch with the sobel edge detection filter
            temp = temp/8                       #Normalisation
            output_matrix[i,j]=temp
    return output_matrix

In [4]:
def laplacian(img):
    patch = [[0, 1, 0], [1, -4, 1], [0, 1 ,0]]  #Laplacian kernel
    patch = np.array(patch)                     # convert the patch to an array

    patch_size = [3,3]                           #define patch size
    img_shape = img.shape                        #image size

    output_matrix = np.zeros([img_shape[0]-2, img_shape[1]-2])  #If image is of size m*n and patch p*q the resultanat of convoluton is of size m-p+1 * n-q+1
    for i in range(img_shape[0]-2):
        for j in range(img_shape[1]-2):
            output = np.zeros(patch_size)       #define a output patch
            for k in range(patch_size[0]):
                for m in range(patch_size[1]):
                    output[k,m]=img[i+k,j+m]    #to extract an output patch
            temp = np.sum(patch*output)      # calculate the sum of product of the output patch with the laplacian kernel
            temp = temp/1     #Normalisation
            output_matrix[i,j]=temp
    return output_matrix
    

In [5]:
def hysteresis(img, low=0.04, high=0.09):  #the high and low values are set deciding tradeoffs. Can vary.
    
    highT = img.max() * high;           #find the value for higher threshold. img.max will give the maximum intensity value in the image which when multiplied by the high argument value gives higher threshold
    lowT = highT * low;                 #find the value for lower threshold. Higher threshold multiplied by the low argument gives lower threshold
    #These threshold values can be set without any above calculations as some random values.
    
    M, N = img.shape                   #img size
    res = np.zeros((M,N), dtype=np.int32)   #array of the original image size
    
    weak = np.int32(25)
    strong = np.int32(255)
    
    strong_i, strong_j = np.where(img >= highT)   #indices with values greater than the higher threshold
    zeros_i, zeros_j = np.where(img < lowT)       #indices with values less than lower threshold.
    
    weak_i, weak_j = np.where((img <= highT) & (img >= lowT))   #indices with values in between the lower and higher threshold
    
    res[strong_i, strong_j] = strong                #make all the indices with value greater than threshold = strong 
    res[weak_i, weak_j] = weak                      #make all the indices with value between the thresholds = weak 
    
    
    #check for the 8-neighbours of a weak pixel, if any one pixel in 8-neighbourhood is strong make it strong.
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (res[i,j] == weak):
                    if ((res[i+1, j-1] == strong) or (res[i+1, j] == strong) or (res[i+1, j+1] == strong)
                        or (res[i, j-1] == strong) or (res[i, j+1] == strong)
                        or (res[i-1, j-1] == strong) or (res[i-1, j] == strong) or (res[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
    
    
    return res

In [6]:
#Read Image
I = cv2.imread('einstein.jfif',0)

# Call for edge detection filter function
Gx = horizontalEdge(I)

# Call for edge detection filter function
Gy = verticalEdge(Gx)

#Call for laplacian function
Laplacian_output = laplacian(Gy)



#Call for hysteresis
final_img = hysteresis(Laplacian_output)

cv2.imwrite('canny.png', final_img)



True